# LSTM for Text Classification

Di notebook ini, kita akan berkenalan dengan salah satu arsitektur *neural networks* yang sering digunakan untuk data sekuensial, yaitu **Long Short-Term Memory (LSTM)**.

## Agenda

Agenda kita hari ini:
* Why LSTM
* LSTM dengan PyTorch
* Bagaimana cara kerja LSTM

In [ ]:
import random
import re
from collections import Counter
from pathlib import Path
from string import punctuation

import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

pd.set_option("display.max_colwidth", 0)

## Why LSTM?

Misalkan kita diberikan potongan berita seperti berikut:

> Timnas Indonesia U-22 lolos ke babak final SEA Games 2019 dengan perjuangan berat

dan juga seperti berikut:

>  Pedangdut Selvi Kitty masih mengupayakan yang terbaik bagi kesembuhan putranya, Abizard Kavin Suseno yang mengidap penyakit langka, sindrom Kawasaki.

Kira-kira 2 berita tersebut masuk ke kategori apa?

---

Setiap kata yang menyusun kalimat di atas, saling terhubung dengan kata sebelum ataupun sesudahnya. Bahkan, mungkin bisa terhubung dengan kata-kata jauh sebelum atau juga setelahnya. Inilah yang menjadi tantangan dalam memproses data teks.

Model seperti CNN ataupun ANN memiliki keterbatasan jika diharuskan belajar dari data sekuensial seperti data teks. Salah satu keterbatasan yang ada dalam deep neural networks atau bahkan convolutional neural networks adalah kedua jenis arsitektur tersebut menerima input vektor dalam bentuk atau ukuran yang tetap (gambar) dan menghasilkan output vektor dengan ukuran yang tetap juga (probabilitas tiap kelas).

## LSTM with PyTorch

Kita akan membuat model LSTM untuk mengklasifikasikan kategori dari artikel berita.

### Datasets

Data yang akan digunakan adalah data artikel yang sudah dikumpulkan yang terdiri dari 9 jenis kategori:
* football
* international news
* health
* politik
* business
* celebs
* local news
* romance
* religi

In [ ]:
DATA_DIR = Path("data/news-article")
DATA_FILEPATH = DATA_DIR / "news.csv"

In [ ]:
df_news = pd.read_csv(DATA_FILEPATH)
df_news = df_news[~df_news["class"].isin(["international_film_tv"])]

In [ ]:
df_news.sample(5)

### Data Preprocessing

In [ ]:
def lowerize(df):
    df["full_text"] = df["full_text"].str.lower()
    return df


def remove_punctuation(df):
    df["full_text"] = df["full_text"].apply(
        lambda excerpt: "".join([char for char in excerpt if char not in punctuation])
    )
    return df


def remove_digits(df):
    df["full_text"] = df["full_text"].apply(
        lambda excerpt: re.sub(r"\b\d+\b", "", excerpt)
    )
    return df


df_news = df_news.pipe(lowerize).pipe(remove_punctuation).pipe(remove_digits)

In [ ]:
df_news.sample(5)

#### Tokenization

In [ ]:
# create dict vocab
count = Counter(" ".join(df_news["full_text"].tolist()).split())
vocab = sorted(count, key=count.get, reverse=False)
vocab_to_int = {word: i for i, word in enumerate(vocab, 1)}
int_to_vocab = {i: word for word, i in vocab_to_int.items()}
print("Number of vocab:", len(vocab))

# tokenize
news_tokens = []
for news in df_news["full_text"]:
    news_tokens.append([vocab_to_int[word] for word in news.split()])

In [ ]:
df_news.loc[1, "full_text"]

In [ ]:
print(news_tokens[1])

In [ ]:
" ".join([int_to_vocab[token] for token in news_tokens[1]])

In [ ]:
news_lengths = Counter([len(x) for x in news_tokens])
print("Maximum news length:", max(news_lengths))
print("Minimum news length:", min(news_lengths))

#### Padding Sequences

In [ ]:
def pad_sequence(sequences, seq_length):
    """
    Return sequences where each sequence is padded with 0's
    or truncated to the `seq_length`
    """
    padded_sequences = np.zeros((len(sequences), seq_length), dtype=int)

    for i, row in enumerate(sequences):
        padded_sequences[i, -len(row):] = np.array(row, dtype=int)[:seq_length]

    return padded_sequences

In [ ]:
SEQ_LENGTH = 500

padded_sequences = pad_sequence(news_tokens, SEQ_LENGTH)

# assert to check all goes as expected
assert len(padded_sequences) == len(news_tokens)
assert len(padded_sequences[0]) == SEQ_LENGTH

In [ ]:
print(padded_sequences[0])

In [ ]:
len(news_tokens[0])

In [ ]:
np.where(padded_sequences == 0, 1, 0)[0].sum()

#### Encode Target Class

In [ ]:
class_to_idx = {c: idx for idx, c in enumerate(df_news["class"].unique())}
df_news["encoded_class"] = df_news["class"].map(class_to_idx)

In [ ]:
print(class_to_idx)

In [ ]:
df_news.sample(5)

### Training, Validation, Test

#### Shuffle Dataset

In [ ]:
features = padded_sequences.copy()
labels = df_news["encoded_class"].values.copy()

rng = np.random.default_rng(11)
rng.shuffle(features)
rng.shuffle(labels)

In [ ]:
TRAIN_SIZE = .8

train_idx = int(len(features) * TRAIN_SIZE)
X_train, X_remaining = features[:train_idx], features[train_idx:]
y_train, y_remaining = labels[:train_idx], labels[train_idx:]

test_idx = int(len(X_remaining) * .5)
X_val, X_test = X_remaining[:test_idx], X_remaining[test_idx:]
y_val, y_test = y_remaining[:test_idx], y_remaining[test_idx:]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{} {}".format(X_train.shape, y_train.shape), 
      "\nValidation set: \t{} {}".format(X_val.shape, y_val.shape),
      "\nTest set: \t\t{} {}".format(X_test.shape, y_test.shape))

#### Data Batching

In [ ]:
BATCH_SIZE = 32

train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
valid_data = TensorDataset(torch.from_numpy(X_val), torch.from_numpy(y_val))
test_data = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))

train_loader = DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
test_loader = DataLoader(test_data, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)

In [ ]:
dataiter = iter(train_loader)
X_sample, y_sample = next(dataiter)

print('Sample input size: ', X_sample.size()) # batch_size, seq_length
print('Sample input: \n', X_sample)
print()
print('Sample label size: ', y_sample.size()) # batch_size
print('Sample label: \n', y_sample)

### Modeling

In [ ]:
is_using_gpu = torch.cuda.is_available()

if is_using_gpu:
    print("Will use GPU for modeling")
else:
    print("No GPU available, will use CPU")

In [ ]:
class NewsLSTM(nn.Module):
    """LSTM model for News Category Classification."""
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=.5):
        """Init model."""
        super().__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim=embedding_dim)
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=n_layers,
            dropout=drop_prob,
            batch_first=True
        )

        # dropout layer
        self.dropout = nn.Dropout(.3)

        # linear and sigmoid layer
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x: torch.Tensor, hidden: torch.Tensor):
        "Perform forward propagation."
        batch_size = x.size()

        # embedding and LSTM outs
        x = x.long()
        embed_outs = self.embedding(x)
        lstm_outs, hidden = self.lstm(embed_outs, hidden)

        # get the last sequence step outputs
        lstm_outs = lstm_outs[:, -1, :]

        # dropout and fully-connected layer
        out = self.dropout(lstm_outs)
        out = self.fc(out)

        # sigmoid
        out = self.sigmoid(out)

        return out, hidden

    def init_hidden(self, batch_size):
        "Init hidden state of LSTM layer."
        # Create two new tensors with size: n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (is_using_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

#### Instantiate LSTM

In [ ]:
len(vocab_to_int)

#### Training Model

In [ ]:
EPOCHS = 4
GRAD_CLIP = 5

In [ ]:
def train_model(
    model, train_loader, valid_loader,
    criterion=criterion, optimizer=optimizer, print_every=50,
    epochs=EPOCHS, grad_clip=GRAD_CLIP, batch_size=BATCH_SIZE
):
    if is_using_gpu:
        model.cuda()

    # set mode to training
    model.train()
    for epoch in range(epochs):
        # initialize hidden state
        h = model.init_hidden(batch_size)

        for counter, (inputs, labels) in enumerate(train_loader, 1):
            if is_using_gpu:
                inputs, labels = inputs.cuda(), labels.cuda()

            # create new instance for hidden state
            # so we don't backprop to the entire training history
            h = tuple([weight.data for weight in h])

            # zeros gradient
            model.zero_grad()

            # forward propagation
            output, h = model(inputs, h)

            # calculate loss and do backprop
            loss = criterion(output.squeeze(), labels.float())
            loss.backward()

            # update weights and clip gradient to avoid exploding gradient
            nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            optimizer.step()

            # validate after `print_every` batch
            if counter % print_every == 0:
                val_h = model.init_hidden(batch_size)
                val_losses = []

                # set mode to validation
                model.eval()
                for inputs, labels in valid_loader:
                    # create new instance for hidden state
                    # so we don't backprop to the entire training history
                    val_h = tuple([weight.data for weight in val_h])

                    if is_using_gpu:
                        inputs, labels = inputs.cuda(), labels.cuda()

                    output, val_h = model(inputs, val_h)
                    val_loss = criterion(output.squeeze(), labels.float())

                    val_losses.append(val_loss.item())

                model.train()
                print("Epoch: {}/{}...".format(epoch, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.6f}...".format(loss.item()),
                      "Val Loss: {:.6f}".format(np.mean(val_losses)))
    return model

In [ ]:
# training components
LEARNING_RATE = 1e-2
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
VOCAB_SIZE = len(vocab_to_int) + 1
OUTPUT_SIZE = 1
EMBEDDING_DIM = 300
HIDDEN_DIM = 128
NUM_LAYERS = 2

# define model
model = NewsLSTM(
    vocab_size=VOCAB_SIZE,
    output_size=OUTPUT_SIZE,
    embedding_dim=EMBEDDING_DIM,
    hidden_dim=HIDDEN_DIM,
    n_layers=NUM_LAYERS
)

model = train_model(model, train_loader, valid_loader)

#### Test Model

In [ ]:
test_losses = []
num_correct = 0

h = model.init_hidden(BATCH_SIZE)

# set mode to eval
model.eval()

for inputs, labels in test_loader:
    h = tuple([weight.data for weight in h])

    if is_using_gpu:
        inputs, labels = inputs.cuda(), labels.cuda()

    output, h = model(inputs, h)

    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())

    prediction = torch.round(output.squeeze())

    correct_prediction = prediction.eq(labels.float().view_as(prediction))
    correct = np.squeeze(correct_prediction.numpy()) if not is_using_gpu else np.squeeze(correct_prediction.cpu().numpy())
    num_correct += np.sum(correct)

# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct / len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

## How LSTM Works?

LSTM sengaja didesain untuk mengatasi permasalahan **long-term dependencies**. Komputasi pada layer LSTM lebih kompleks jika dibandingkan dengan model *neural network* berjenis sekuensial lainnya (misal RNN). Ini dikarenakan LSTM mencoba mempertahankan informasi yang diterima jauh sebelum data saat ini diproses.

LSTM menghasilkan 2 output:
* long-term memory
* short-term memory

Komputasi pada layer LSTM melibatkan **2 fungsi aktivasi** sesuai dengan fungsi untuk menghasilkan long-term memory dan juga short-term memory.

<div align='center'>
     <img src="https://sylabs.notion.site/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F39eea92c-e227-4707-b28c-d377461a528e%2Flstm-diagram.png?table=block&id=b5653706-1884-4ec8-9b06-2cea7564e4fa&spaceId=685593da-9b2b-4a94-b296-d52808c79757&width=2000&userId=&cache=v2" width=70%/>
</div>

Jika kita bayangkan 2 input dan 2 output pada layer LSTM sebagai **“gerbang”** masuk dan keluar layer, maka layer LSTM memiliki 4 “gerbang” komputasi, yaitu:
* learn gate
* forget gate
* remember gate
* use gate